# NV Regulatory Data

Preprocessing input data for a smoother upload experience of the state data to the WaDE 2.0 database.
Using geopandas to read in shp file, and coverting to WKT for ReportingUnit geometry.

Notes:
- WKT will be in it's own separate geometery dataframe.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Nevada/Regulatory/RawInputData"
os.chdir(workingDir)

In [2]:
#CSV input file
fileInput = "SE_GroundwaterBasins_DesigOrders_input.csv"

df = pd.read_csv(fileInput)
print(len(df))
df.head()

282


,OID_,BasinID,BasinName,SubAreaNam,Designated,DesigStatu,DesigOrder,PreferUse,HydRegID,HydRegName,Shape_STAr,Shape_STLe
0,0,009,Long Valley,,No,,,,1,Northwest Region,1.126936e+09,225645.674761
1,1,033B,Quinn River Valley,McDermitt Subarea,No,,,,2,Black Rock Desert Region,2.396199e+08,247552.099201
2,2,012,Mosquito Valley,,No,,,,1,Northwest Region,8.575338e+07,47416.631321
3,3,007,Swan Lake Valley,,No,,,,1,Northwest Region,6.051536e+08,140437.181358
4,4,029,Pine Forest Valley,,Yes,PUID,711,1290,2,Black Rock Desert Region,1.370977e+09,191394.527411


In [3]:
# Update BasinName to include "Designated" into name for duplicate entries.
def updateBasinName(valA, valB):
    if valB == "No":
        outString = valA
    else:
        outString = valA + " Designated"

    return outString

df['in_BasinName'] = df.apply(lambda row: updateBasinName(row['BasinName'], row['Designated']), axis=1)
df.head()

,OID_,BasinID,BasinName,SubAreaNam,Designated,DesigStatu,DesigOrder,PreferUse,HydRegID,HydRegName,Shape_STAr,Shape_STLe,in_BasinName
0,0,009,Long Valley,,No,,,,1,Northwest Region,1.126936e+09,225645.674761,Long Valley
1,1,033B,Quinn River Valley,McDermitt Subarea,No,,,,2,Black Rock Desert Region,2.396199e+08,247552.099201,Quinn River Valley
2,2,012,Mosquito Valley,,No,,,,1,Northwest Region,8.575338e+07,47416.631321,Mosquito Valley
3,3,007,Swan Lake Valley,,No,,,,1,Northwest Region,6.051536e+08,140437.181358,Swan Lake Valley
4,4,029,Pine Forest Valley,,Yes,PUID,711,1290,2,Black Rock Desert Region,1.370977e+09,191394.527411,Pine Forest Valley Designated


In [4]:
# Include RegulatoryStatusCV

def includeRegulatoryStatusCV(valA):
    valA = valA.strip()
    if valA == "No":
        outString = "Not Designated"
    else:
        outString = "Designated"

    return outString

df['in_RegulatoryStatusCV'] = df.apply(lambda row: includeRegulatoryStatusCV(row['Designated']), axis=1)
df.head()

,OID_,BasinID,BasinName,SubAreaNam,Designated,DesigStatu,DesigOrder,PreferUse,HydRegID,HydRegName,Shape_STAr,Shape_STLe,in_BasinName,in_RegulatoryStatusCV
0,0,009,Long Valley,,No,,,,1,Northwest Region,1.126936e+09,225645.674761,Long Valley,Not Designated
1,1,033B,Quinn River Valley,McDermitt Subarea,No,,,,2,Black Rock Desert Region,2.396199e+08,247552.099201,Quinn River Valley,Not Designated
2,2,012,Mosquito Valley,,No,,,,1,Northwest Region,8.575338e+07,47416.631321,Mosquito Valley,Not Designated
3,3,007,Swan Lake Valley,,No,,,,1,Northwest Region,6.051536e+08,140437.181358,Swan Lake Valley,Not Designated
4,4,029,Pine Forest Valley,,Yes,PUID,711,1290,2,Black Rock Desert Region,1.370977e+09,191394.527411,Pine Forest Valley Designated,Designated


In [5]:
# Include RegulatoryDescription

UnknownDescription = {
"D" : "State Engineer's order(s) do not define any administrative controls.",
"ID" : "State Engineer's order(s) state that irrigation is not a preferred use in these basins and applications for new irrigation appropriations will be denied.",
"PU" : "State Engineer's order(s) list certain types of uses as preferred in these basins, and quantity restrictions may be placed on these preferred uses.",
"PUID" : "State Engineer's order(s) list certain types of uses as preferred in these basins.  Quantity restrictions may be placed on these preferred uses.  State Engineer's order(s) also state that irrigation is not a preferred use in these basins and applications for new irrigation appropriations will be denied.  Other uses may also be listed as denied."}

def assignUnknownDescription(valA):
    valA = valA.strip()
    if valA == "" or pd.isnull(valA):
        outString = "No description, not designated."
    else:
        outString = UnknownDescription[valA]
    return outString

df['in_RegulatoryDescription'] = df.apply(lambda row: assignUnknownDescription(row['DesigStatu']), axis=1)
df.head()

,OID_,BasinID,BasinName,SubAreaNam,Designated,DesigStatu,DesigOrder,PreferUse,HydRegID,HydRegName,Shape_STAr,Shape_STLe,in_BasinName,in_RegulatoryStatusCV,in_RegulatoryDescription
0,0,009,Long Valley,,No,,,,1,Northwest Region,1.126936e+09,225645.674761,Long Valley,Not Designated,"No description, not designated."
1,1,033B,Quinn River Valley,McDermitt Subarea,No,,,,2,Black Rock Desert Region,2.396199e+08,247552.099201,Quinn River Valley,Not Designated,"No description, not designated."
2,2,012,Mosquito Valley,,No,,,,1,Northwest Region,8.575338e+07,47416.631321,Mosquito Valley,Not Designated,"No description, not designated."
3,3,007,Swan Lake Valley,,No,,,,1,Northwest Region,6.051536e+08,140437.181358,Swan Lake Valley,Not Designated,"No description, not designated."
4,4,029,Pine Forest Valley,,Yes,PUID,711,1290,2,Black Rock Desert Region,1.370977e+09,191394.527411,Pine Forest Valley Designated,Designated,State Engineer's order(s) list certain types o...


In [6]:
df_out = df

# Shapefile Data

In [7]:
# Interstate Stream Compact Regions

ShapeFile = "ShapefileData/SE_GroundwaterBasins_DesigOrders/SE_GroundwaterBasins_DesigOrders.shp"
dfshp = gpd.read_file(ShapeFile)

print(len(dfshp))
dfshp.head()

282


,BasinID,BasinName,SubAreaNam,Designated,DesigStatu,DesigOrder,PreferUse,HydRegID,HydRegName,Shape_STAr,Shape_STLe,geometry
0,009,Long Valley,None,No,None,None,None,1,Northwest Region,1.126936e+09,225645.674761,"POLYGON ((275769.417 4649545.137, 275801.697 4..."
1,033B,Quinn River Valley,McDermitt Subarea,No,None,None,None,2,Black Rock Desert Region,2.396199e+08,247552.099201,"MULTIPOLYGON (((456894.477 4649809.937, 456869..."
2,012,Mosquito Valley,None,No,None,None,None,1,Northwest Region,8.575338e+07,47416.631321,"POLYGON ((272861.427 4639227.637, 272832.477 4..."
3,007,Swan Lake Valley,None,No,None,None,None,1,Northwest Region,6.051536e+08,140437.181358,"POLYGON ((306578.287 4638189.407, 306923.487 4..."
4,029,Pine Forest Valley,None,Yes,PUID,711,1290,2,Black Rock Desert Region,1.370977e+09,191394.527411,"POLYGON ((383016.817 4637824.387, 383964.507 4..."


In [8]:
# Interstate Stream Compact Regions output shapefile dataframe
columnList = ['BasinID', 
              'BasinName',
              'geometry']
dfshp_out = pd.DataFrame(columns=columnList, index=dfshp.index)

dfshp_out['BasinID'] = dfshp['BasinID']
dfshp_out['BasinName'] = dfshp['BasinName'].str.title()
dfshp_out['geometry'] = dfshp['geometry']

print(len(dfshp_out))
dfshp_out.head(3)

282


,BasinID,BasinName,geometry
0,009,Long Valley,"POLYGON ((275769.417 4649545.137, 275801.697 4..."
1,033B,Quinn River Valley,"MULTIPOLYGON (((456894.477 4649809.937, 456869..."
2,012,Mosquito Valley,"POLYGON ((272861.427 4639227.637, 272832.477 4..."


### Inspect Output Data & Export

In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_out.dtypes)

OID_                          int64
BasinID                      object
BasinName                    object
SubAreaNam                   object
Designated                   object
DesigStatu                   object
DesigOrder                   object
PreferUse                    object
HydRegID                      int64
HydRegName                   object
Shape_STAr                  float64
Shape_STLe                  float64
in_BasinName                 object
in_RegulatoryStatusCV        object
in_RegulatoryDescription     object
dtype: object


In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfshp_out.dtypes)

BasinID        object
BasinName      object
geometry     geometry
dtype: object


In [11]:
# Export out to CSV.
df_out.to_csv('P_nvRegMaster.csv', index=False) # The output.
dfshp_out.to_csv('P_nvRegGeometry.csv', index=False) # The output geometry.